In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

import json # library to handle JSON files

import requests # library to handle requests

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import matplotlib.cm as cm
import matplotlib.colors as colors # Matplotlib and associated plotting modules

from sklearn.cluster import KMeans # import k-means from clustering stage

!conda install -c conda-forge beautifulsoup4 --yes
from bs4 import BeautifulSoup # website scraping libraries and packages in Python from BeautifulSoup 

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim  # convert an address into latitude and longitude values

print("Libraries imported.")

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.2
  latest version: 4.8.4

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /srv/conda/envs/notebook

  added / updated specs:
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.9.1       |   py36h9f0ad1d_0         163 KB  conda-forge
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    soupsieve-2.0.1            |   py36h9f0ad1d_0          56 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.3 MB

The following NEW packages will be INSTALLED:

  beautifulsoup4     conda-forge/linux-64::beautifulsoup4-4.9.1-py36h9f0ad1d_0
  soupsieve          conda-forge/linux-64::soupsiev

In [3]:
!pip install folium
import folium # map rendering library

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

     |████████████████████████████████| 93 kB 1.2 MB/s eta 0:00:011


In [4]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [5]:
soup = BeautifulSoup(data, 'html.parser')

In [6]:
postalCodeList = []
boroughList = []
neighborhoodList = []

In [7]:

soup.find('table').find_all('tr')

# find all the rows of the table
soup.find('table').find_all('tr')

# for each row of the table, find all the table data
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')

In [9]:
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text.rstrip('\n'))

In [10]:
toronto_df = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})

toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned
1,M2A\n,Not assigned\n,Not assigned
2,M3A\n,North York\n,Parkwoods
3,M4A\n,North York\n,Victoria Village
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront"


In [11]:
toronto_df_drop = toronto_df[toronto_df.Borough != "Not assigned"].reset_index(drop=True)
toronto_df_drop.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned
1,M2A\n,Not assigned\n,Not assigned
2,M3A\n,North York\n,Parkwoods
3,M4A\n,North York\n,Victoria Village
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront"


In [12]:
toronto_df_grouped = toronto_df_drop.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned
1,M1B\n,Scarborough\n,"Malvern, Rouge"
2,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek"
3,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill"
4,M1G\n,Scarborough\n,Woburn


In [13]:
for index, row in toronto_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M1B\n,Scarborough\n,"Malvern, Rouge"
2,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek"
3,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill"
4,M1G\n,Scarborough\n,Woburn


In [14]:
column_names = ["PostalCode", "Borough", "Neighborhood"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_grouped[toronto_df_grouped["PostalCode"]==postcode], ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood


In [15]:
test_df.head()

,PostalCode,Borough,Neighborhood


In [16]:
toronto_df_grouped.shape

(180, 3)

In [17]:
coordinates = pd.read_csv('https://cocl.us/Geospatial_data')
coordinates.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [19]:
toronto_df_new = toronto_df_grouped.merge(coordinates, on="PostalCode", how="left")
toronto_df_new.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1A\n,Not assigned\n,Not assigned\n,NaN,NaN
1,M1B\n,Scarborough\n,"Malvern, Rouge",NaN,NaN
2,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek",NaN,NaN
3,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill",NaN,NaN
4,M1G\n,Scarborough\n,Woburn,NaN,NaN


In [ ]:
column_names = ["PostalCode", "Borough", "Neighborhood", "Latitude", "Longitude"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_new[toronto_df_new["PostalCode"]==postcode], ignore_index=True)
    
test_df

In [22]:
address = 'Toronto'

geolocator = Nominatim(user_agent="supraja")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [ ]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto
map.save("map_toronto.html")

**Exploring neighborhoods in Toronto**

In [26]:
borough_names = list(toronto_df_new.Borough.unique())

borough_with_toronto = []

for x in borough_names:
    if "toronto" in x.lower():
        borough_with_toronto.append(x)
        
borough_with_toronto

['East Toronto\n', 'Central Toronto\n', 'Downtown Toronto\n', 'West Toronto\n']

In [32]:
toronto_df_new = toronto_df_new[toronto_df_new['Borough'].isin(borough_with_toronto)].reset_index(drop=True)
print(toronto_df_new.shape)
toronto_df_new.head()

(39, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E\n,East Toronto\n,The Beaches,NaN,NaN
1,M4K\n,East Toronto\n,"The Danforth West, Riverdale",NaN,NaN
2,M4L\n,East Toronto\n,"India Bazaar, The Beaches West",NaN,NaN
3,M4M\n,East Toronto\n,Studio District,NaN,NaN
4,M4N\n,Central Toronto\n,Lawrence Park,NaN,NaN


In [34]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7.add_to(map_toronto)  
    
map_toronto

SyntaxError: invalid syntax (<ipython-input-34-e6f86abb1a88>, line 16)

**Define Foursquare Credentials and Version**

In [35]:
CLIENT_ID = 'LLSABZTE4AXRGDRATVV3EQXTQXZKJX1DZBI052EEVAVYCCQJ' # your Foursquare ID
CLIENT_SECRET = 'X1PKIVIPIYY3ILAKQG3XXYZD4SFJCP1ON14L4AV00LRHQRS1'  # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LLSABZTE4AXRGDRATVV3EQXTQXZKJX1DZBI052EEVAVYCCQJ
CLIENT_SECRET:X1PKIVIPIYY3ILAKQG3XXYZD4SFJCP1ON14L4AV00LRHQRS1


In [36]:
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['PostalCode'], toronto_df_new['Borough'], 
                                                  toronto_df_new['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id=LLSABZTE4AXRGDRATVV3EQXTQXZKJX1DZBI052EEVAVYCCQJ&client_secret=X1PKIVIPIYY3ILAKQG3XXYZD4SFJCP1ON14L4AV00LRHQRS1&v=20180605 \
     &ll=43.653963,-79.387207&radius=500&limit=100".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [37]:
venues_df = pd.DataFrame(venues)


venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(1872, 9)


,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M4E\n,East Toronto\n,The Beaches,NaN,NaN,Downtown Toronto,43.653232,-79.385296,Neighborhood
1,M4E\n,East Toronto\n,The Beaches,NaN,NaN,Japango,43.655268,-79.385165,Sushi Restaurant
2,M4E\n,East Toronto\n,The Beaches,NaN,NaN,Poke Guys,43.654895,-79.385052,Poke Place
3,M4E\n,East Toronto\n,The Beaches,NaN,NaN,Cafe Plenty,43.654571,-79.389450,Café
4,M4E\n,East Toronto\n,The Beaches,NaN,NaN,Textile Museum of Canada,43.654396,-79.386500,Art Museum


In [38]:
venues_df.groupby(["PostalCode", "Borough", "Neighborhood"]).count()

,,,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
PostalCode,Borough,Neighborhood,,,,,,
M4E\n,East Toronto\n,The Beaches,0,0,48,48,48,48
M4K\n,East Toronto\n,"The Danforth West, Riverdale",0,0,48,48,48,48
M4L\n,East Toronto\n,"India Bazaar, The Beaches West",0,0,48,48,48,48
M4M\n,East Toronto\n,Studio District,0,0,48,48,48,48
M4N\n,Central Toronto\n,Lawrence Park,0,0,48,48,48,48
M4P\n,Central Toronto\n,Davisville North,0,0,48,48,48,48
M4R\n,Central Toronto\n,"North Toronto West, Lawrence Park",0,0,48,48,48,48
M4S\n,Central Toronto\n,Davisville,0,0,48,48,48,48
M4T\n,Central Toronto\n,"Moore Park, Summerhill East",0,0,48,48,48,48


In [39]:
toronto_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
toronto_onehot['PostalCode'] = venues_df['PostalCode'] 
toronto_onehot['Borough'] = venues_df['Borough'] 
toronto_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(1872, 41)


,PostalCode,Borough,Neighborhoods,Art Gallery,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop,Café,...,Poke Place,Pub,Salon / Barbershop,Seafood Restaurant,Smoke Shop,Steakhouse,Sushi Restaurant,Toy / Game Store,University,Vegetarian / Vegan Restaurant
0,M4E\n,East Toronto\n,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M4E\n,East Toronto\n,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,M4E\n,East Toronto\n,The Beaches,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,M4E\n,East Toronto\n,The Beaches,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,M4E\n,East Toronto\n,The Beaches,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
toronto_grouped = toronto_onehot.groupby(["PostalCode", "Borough", "Neighborhoods"]).mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped

(39, 41)


,PostalCode,Borough,Neighborhoods,Art Gallery,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop,Café,...,Poke Place,Pub,Salon / Barbershop,Seafood Restaurant,Smoke Shop,Steakhouse,Sushi Restaurant,Toy / Game Store,University,Vegetarian / Vegan Restaurant
0,M4E\n,East Toronto\n,The Beaches,0.0625,0.020833,0.020833,0.020833,0.041667,0.020833,0.0625,...,0.020833,0.020833,0.020833,0.020833,0.020833,0.020833,0.041667,0.020833,0.020833,0.020833
1,M4K\n,East Toronto\n,"The Danforth West, Riverdale",0.0625,0.020833,0.020833,0.020833,0.041667,0.020833,0.0625,...,0.020833,0.020833,0.020833,0.020833,0.020833,0.020833,0.041667,0.020833,0.020833,0.020833
2,M4L\n,East Toronto\n,"India Bazaar, The Beaches West",0.0625,0.020833,0.020833,0.020833,0.041667,0.020833,0.0625,...,0.020833,0.020833,0.020833,0.020833,0.020833,0.020833,0.041667,0.020833,0.020833,0.020833
3,M4M\n,East Toronto\n,Studio District,0.0625,0.020833,0.020833,0.020833,0.041667,0.020833,0.0625,...,0.020833,0.020833,0.020833,0.020833,0.020833,0.020833,0.041667,0.020833,0.020833,0.020833
4,M4N\n,Central Toronto\n,Lawrence Park,0.0625,0.020833,0.020833,0.020833,0.041667,0.020833,0.0625,...,0.020833,0.020833,0.020833,0.020833,0.020833,0.020833,0.041667,0.020833,0.020833,0.020833
5,M4P\n,Central Toronto\n,Davisville North,0.0625,0.020833,0.020833,0.020833,0.041667,0.020833,0.0625,...,0.020833,0.020833,0.020833,0.020833,0.020833,0.020833,0.041667,0.020833,0.020833,0.020833
6,M4R\n,Central Toronto\n,"North Toronto West, Lawrence Park",0.0625,0.020833,0.020833,0.020833,0.041667,0.020833,0.0625,...,0.020833,0.020833,0.020833,0.020833,0.020833,0.020833,0.041667,0.020833,0.020833,0.020833
7,M4S\n,Central Toronto\n,Davisville,0.0625,0.020833,0.020833,0.020833,0.041667,0.020833,0.0625,...,0.020833,0.020833,0.020833,0.020833,0.020833,0.020833,0.041667,0.020833,0.020833,0.020833
8,M4T\n,Central Toronto\n,"Moore Park, Summerhill East",0.0625,0.020833,0.020833,0.020833,0.041667,0.020833,0.0625,...,0.020833,0.020833,0.020833,0.020833,0.020833,0.020833,0.041667,0.020833,0.020833,0.020833
9,M4V\n,Central Toronto\n,"Summerhill West, Rathnelly, South Hill, Forest...",0.0625,0.020833,0.020833,0.020833,0.041667,0.020833,0.0625,...,0.020833,0.020833,0.020833,0.020833,0.020833,0.020833,0.041667,0.020833,0.020833,0.020833


In [41]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    row_categories = toronto_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted

(39, 13)


,PostalCode,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E\n,East Toronto\n,The Beaches,Art Gallery,Café,Coffee Shop,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Hotel,Arts & Crafts Store,Bar,Bubble Tea Shop
1,M4K\n,East Toronto\n,"The Danforth West, Riverdale",Art Gallery,Café,Coffee Shop,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Hotel,Arts & Crafts Store,Bar,Bubble Tea Shop
2,M4L\n,East Toronto\n,"India Bazaar, The Beaches West",Art Gallery,Café,Coffee Shop,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Hotel,Arts & Crafts Store,Bar,Bubble Tea Shop
3,M4M\n,East Toronto\n,Studio District,Art Gallery,Café,Coffee Shop,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Hotel,Arts & Crafts Store,Bar,Bubble Tea Shop
4,M4N\n,Central Toronto\n,Lawrence Park,Art Gallery,Café,Coffee Shop,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Hotel,Arts & Crafts Store,Bar,Bubble Tea Shop
5,M4P\n,Central Toronto\n,Davisville North,Art Gallery,Café,Coffee Shop,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Hotel,Arts & Crafts Store,Bar,Bubble Tea Shop
6,M4R\n,Central Toronto\n,"North Toronto West, Lawrence Park",Art Gallery,Café,Coffee Shop,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Hotel,Arts & Crafts Store,Bar,Bubble Tea Shop
7,M4S\n,Central Toronto\n,Davisville,Art Gallery,Café,Coffee Shop,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Hotel,Arts & Crafts Store,Bar,Bubble Tea Shop
8,M4T\n,Central Toronto\n,"Moore Park, Summerhill East",Art Gallery,Café,Coffee Shop,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Hotel,Arts & Crafts Store,Bar,Bubble Tea Shop
9,M4V\n,Central Toronto\n,"Summerhill West, Rathnelly, South Hill, Forest...",Art Gallery,Café,Coffee Shop,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Hotel,Arts & Crafts Store,Bar,Bubble Tea Shop


**CLUSTERING**

In [42]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop(["PostalCode", "Borough", "Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

/srv/conda/envs/notebook/lib/python3.6/site-packages/ipykernel_launcher.py:6: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [43]:
toronto_merged = toronto_df_new.copy()

# add clustering labels
toronto_merged["Cluster Labels"] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.drop(["Borough", "Neighborhoods"], 1).set_index("PostalCode"), on="PostalCode")

print(toronto_merged.shape)
toronto_merged.head() 

(39, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E\n,East Toronto\n,The Beaches,NaN,NaN,0,Art Gallery,Café,Coffee Shop,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Hotel,Arts & Crafts Store,Bar,Bubble Tea Shop
1,M4K\n,East Toronto\n,"The Danforth West, Riverdale",NaN,NaN,0,Art Gallery,Café,Coffee Shop,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Hotel,Arts & Crafts Store,Bar,Bubble Tea Shop
2,M4L\n,East Toronto\n,"India Bazaar, The Beaches West",NaN,NaN,0,Art Gallery,Café,Coffee Shop,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Hotel,Arts & Crafts Store,Bar,Bubble Tea Shop
3,M4M\n,East Toronto\n,Studio District,NaN,NaN,0,Art Gallery,Café,Coffee Shop,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Hotel,Arts & Crafts Store,Bar,Bubble Tea Shop
4,M4N\n,Central Toronto\n,Lawrence Park,NaN,NaN,0,Art Gallery,Café,Coffee Shop,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Hotel,Arts & Crafts Store,Bar,Bubble Tea Shop


In [44]:
print(toronto_merged.shape)
toronto_merged.sort_values(["Cluster Labels"], inplace=True)
toronto_merged

(39, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E\n,East Toronto\n,The Beaches,NaN,NaN,0,Art Gallery,Café,Coffee Shop,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Hotel,Arts & Crafts Store,Bar,Bubble Tea Shop
21,M5L\n,Downtown Toronto\n,"Commerce Court, Victoria Hotel",NaN,NaN,0,Art Gallery,Café,Coffee Shop,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Hotel,Arts & Crafts Store,Bar,Bubble Tea Shop
22,M5N\n,Central Toronto\n,Roselawn,NaN,NaN,0,Art Gallery,Café,Coffee Shop,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Hotel,Arts & Crafts Store,Bar,Bubble Tea Shop
23,M5P\n,Central Toronto\n,"Forest Hill North & West, Forest Hill Road Park",NaN,NaN,0,Art Gallery,Café,Coffee Shop,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Hotel,Arts & Crafts Store,Bar,Bubble Tea Shop
24,M5R\n,Central Toronto\n,"The Annex, North Midtown, Yorkville",NaN,NaN,0,Art Gallery,Café,Coffee Shop,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Hotel,Arts & Crafts Store,Bar,Bubble Tea Shop
25,M5S\n,Downtown Toronto\n,"University of Toronto, Harbord",NaN,NaN,0,Art Gallery,Café,Coffee Shop,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Hotel,Arts & Crafts Store,Bar,Bubble Tea Shop
26,M5T\n,Downtown Toronto\n,"Kensington Market, Chinatown, Grange Park",NaN,NaN,0,Art Gallery,Café,Coffee Shop,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Hotel,Arts & Crafts Store,Bar,Bubble Tea Shop
27,M5V\n,Downtown Toronto\n,"CN Tower, King and Spadina, Railway Lands, Har...",NaN,NaN,0,Art Gallery,Café,Coffee Shop,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Hotel,Arts & Crafts Store,Bar,Bubble Tea Shop
20,M5K\n,Downtown Toronto\n,"Toronto Dominion Centre, Design Exchange",NaN,NaN,0,Art Gallery,Café,Coffee Shop,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Hotel,Arts & Crafts Store,Bar,Bubble Tea Shop
28,M5W\n,Downtown Toronto\n,Stn A PO Boxes,NaN,NaN,0,Art Gallery,Café,Coffee Shop,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Hotel,Arts & Crafts Store,Bar,Bubble Tea Shop


In [51]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['PostalCode'], toronto_merged['Borough'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7.add_to(map_clusters)
       
map_clusters

SyntaxError: invalid syntax (<ipython-input-51-89dae1819610>, line 22)

**CHECKING THE CLUSTERS**

**Cluster 1**

In [46]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + \
                                                                                 list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto\n,0,Art Gallery,Café,Coffee Shop,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Hotel,Arts & Crafts Store,Bar,Bubble Tea Shop
21,Downtown Toronto\n,0,Art Gallery,Café,Coffee Shop,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Hotel,Arts & Crafts Store,Bar,Bubble Tea Shop
22,Central Toronto\n,0,Art Gallery,Café,Coffee Shop,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Hotel,Arts & Crafts Store,Bar,Bubble Tea Shop
23,Central Toronto\n,0,Art Gallery,Café,Coffee Shop,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Hotel,Arts & Crafts Store,Bar,Bubble Tea Shop
24,Central Toronto\n,0,Art Gallery,Café,Coffee Shop,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Hotel,Arts & Crafts Store,Bar,Bubble Tea Shop
25,Downtown Toronto\n,0,Art Gallery,Café,Coffee Shop,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Hotel,Arts & Crafts Store,Bar,Bubble Tea Shop
26,Downtown Toronto\n,0,Art Gallery,Café,Coffee Shop,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Hotel,Arts & Crafts Store,Bar,Bubble Tea Shop
27,Downtown Toronto\n,0,Art Gallery,Café,Coffee Shop,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Hotel,Arts & Crafts Store,Bar,Bubble Tea Shop
20,Downtown Toronto\n,0,Art Gallery,Café,Coffee Shop,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Hotel,Arts & Crafts Store,Bar,Bubble Tea Shop
28,Downtown Toronto\n,0,Art Gallery,Café,Coffee Shop,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Hotel,Arts & Crafts Store,Bar,Bubble Tea Shop


**Cluster 2**

In [47]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + \
                                                                                 list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


**Cluster 3**

In [48]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + \
                                                                                 list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


**Cluster 4**

In [49]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + \
                                                                                 list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


**Cluster 5**

In [50]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + \
                                                                                 list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


**CONCLUSION**

Most of the neighborhoods fall into Cluster 1 which are the areas with cafe, restaurants, supermarkets etc